### 基于 SQL 数据构建问答系统

使 LLM 系统能够查询结构化数据可能与非结构化文本数据有质的不同。

在后者中，通常生成可以针对向量数据库进行搜索的文本，而结构化数据的方法通常是让 LLM 在 DSL（例如 SQL）中编写和执行查询。

在本指南中，我们将介绍在数据库中的表格数据上创建问答系统的基本方法。

我们将介绍使用链和代理的实现。

这些系统将允许我们提出有关数据库中数据的问题，并得到自然语言的答案。

两者之间的主要区别在于，我们的代理可以在循环中多次查询数据库以回答问题。

#### 安全说明 

构建 SQL 数据库的 Q&A 系统需要执行模型生成的 SQL 查询。这样做存在固有的风险。

确保您的数据库连接权限的范围始终尽可能窄，以满足您的链/代理的需求。这将减轻（但不能消除）构建模型驱动系统的风险。

有关常规安全最佳做法的详细信息，请参阅此处。

#### Build 构建

概括地说，这些系统的步骤是：

将问题转换为 DSL 查询：模型将用户输入转换为 SQL 查询。

执行 SQL 查询：执行查询。

回答问题：模型使用查询结果响应用户输入。

请注意，在 CSV 中查询数据可以遵循类似的方法。

有关详细信息，请参阅我们关于对 CSV 数据进行问答的操作指南。

![Alternative text](https://python.langchain.com/v0.2/assets/images/sql_usecase-d432701261f05ab69b38576093718cf3.png "optional title")

### 设置
首先，获取所需的包并设置环境变量：

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

#### 使用Sqlite
将使用与 Chinook 数据库的 SQLite 连接。按照以下安装步骤在与此笔记本相同的目录中创建：`Chinook.db`

将此文件另存为`Chinook.sql`

跑`sqlite3 Chinook.db`

跑`.read Chinook.sql`

测试`SELECT * FROM Artist LIMIT 10;`

现在，在我们的目录中，我们可以使用 SQLAlchemy 驱动的类与它进行交互：`Chinhook.dbSQLDatabase`

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

很棒，我们有一个可以查询的 SQL 数据库。现在让我们尝试将它连接到 LLM。

### Chains 链

链（即 LangChain Runnables 的组成）支持其步骤可预测的应用程序。

我们可以创建一个简单的链来回答一个问题并执行以下操作：

- 将问题转换为 SQL 查询;
- 执行查询;
- 使用结果来回答原始问题。

此安排不支持某些方案。

例如，此系统将对任何用户输入执行 SQL 查询，甚至是“hello”。

重要的是，正如我们将在下面看到的，有些问题需要多个查询才能回答。

我们将在代理部分中介绍这些方案。

#### 将问题转换为 SQL 查询

SQL 链或代理的第一步是获取用户输入并将其转换为 SQL 查询。

LangChain为此提供了一个内置链：`create_sql_query_chain`

In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")  # 这次，我们还是用阿里

In [6]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "有多少员工?"})
response

'SELECT COUNT(*) FROM "Employee"'

In [7]:
# 我们可以执行查询以确保它有效：
db.run(response)

'[(8,)]'

我们可以查看 LangSmith 跟踪，以更好地了解该链在做什么。

我们还可以直接检查链条的提示。查看提示（如下），我们可以看到它是：

特定于方言。在本例中，它显式引用 SQLite。

具有所有可用表的定义。


每个表有三个示例行。
这种技术的灵感来自这样的论文，这些论文建议显示示例行并明确表可以提高性能。

我们也可以像这样检查完整的提示：

In [8]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

#### 执行 SQL 查询
现在我们已经生成了一个 SQL 查询，我们将需要执行它。

这是创建 SQL 链最危险的部分。请仔细考虑是否可以对数据运行自动查询。

尽可能减少数据库连接权限。请考虑在执行查询之前向链添加人工审批步骤（见下文）。

我们可以使用 来轻松地将查询执行添加到我们的链中：`QuerySQLDatabaseTool`

In [9]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "有多少位员工?"})

'[(8,)]'

#### 回答问题
现在我们已经有了自动生成和执行查询的方法，我们只需要将原始问题和 SQL 查询结果结合起来即可生成最终答案。

我们可以通过再次将问题和结果传递给 LLM 来做到这一点：

In [10]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """给定以下用户问题、相应的 SQL 查询和 SQL 结果，回答用户问题.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "有多少位员工?"})

'有8位员工。'

让我们回顾一下上述 LCEL 中发生的事情。假设此链被调用。

在第一个之后，我们有一个具有两个元素的可运行对象：

Where 将生成一个 SQL 查询来回答问题。`RunnablePassthrough.assign` `{"question": question, "query": write_query.invoke(question)}` `write_query`

在第二个 之后，我们添加了第三个元素，其中包含 ，其中在上一步中计算。`RunnablePassthrough.assign` `"result"`  `execute_query.invoke(query)query`

这三个输入被格式化到提示符中并传递到 LLM 中。

提取输出消息的字符串内容。`StrOutputParser()`

请注意，我们将 LLM、工具、提示和其他链组合在一起，但由于每个链都实现了 Runnable 接口，因此它们的输入和输出可以以合理的方式绑定在一起。

#### 后续步骤
对于更复杂的查询生成，我们可能希望创建少量提示或添加查询检查步骤。
有关此类高级技术以及更多内容，请查看：

1. 提示策略：高级提示工程技术。

2. 查询检查：增加查询验证和错误处理。

3. 大型数据库：使用大型数据库的技术。

### Agent 代理

LangChain 有一个 SQL 代理，它提供了一种比链更灵活的与 SQL 数据库交互的方式。使用 SQL 代理的主要优点是：

它可以根据数据库的架构以及数据库的内容（如描述特定表）回答问题。

它可以通过运行生成的查询、捕获回溯并正确重新生成它来从错误中恢复。

它可以根据需要多次查询数据库以回答用户问题。

它将仅通过从相关表中检索架构来保存令牌。

为了初始化代理，我们将使用 创建一堆工具：`SQLDatabaseToolkit`

- 创建和执行查询
- 检查查询语法
- 检索表说明
- ...和更多

In [11]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016F01B7FD50>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016F01B7FD50>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016F01B7FD50>),
 QuerySQLCheckerTool(description='Use this tool to 

### System Prompt  提示词

我们还需要为我们的代理创建一个系统提示。这将包括有关如何表现的说明

In [12]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """您是设计用于与 SQL 数据库交互的代理。
给定一个输入问题，创建一个语法正确的 SQLite 查询来运行，然后查看查询结果并返回答案。
除非用户指定了他们希望获取的特定数量的示例，否则请始终将查询限制为最多 5 个结果。
您可以按相关列对结果进行排序，以返回数据库中最有趣的示例。
永远不要查询特定表中的所有列，只要求给定问题的相关列。
您可以访问用于与数据库交互的工具。
仅使用以下工具。仅使用以下工具返回的信息来构建您的最终答案。
在执行查询之前，必须仔细检查查询。如果在执行查询时遇到错误，请重写查询，然后重试。

不要对数据库进行任何 DML 语句 (INSERT、UPDATE、DELETE、DROP 等)。

首先，您应该始终查看数据库中的表，以了解可以查询的内容。
请勿跳过此步骤。
然后，您应该查询最相关表的架构。."""

system_message = SystemMessage(content=SQL_PREFIX)

### init agent 初始化代理

In [ ]:
# 首先，获取所需的包 LangGraph
%pip install --upgrade --quiet langgraph

In [13]:
# 我们将使用预构建的 LangGraph 代理来构建我们的代理

from langchain_core.messages import HumanMessage
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm, tools, messages_modifier=system_message
)

考虑代理如何回答以下问题

In [20]:
try:    
    for s in agent_executor.stream(
        {"messages": [HumanMessage(content="哪个国家的客户花费最多?")]}
    ):
        print(s)
        print("----")
except Exception as e:
    print(f"An error occurred: {e}")



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 788, 'total_tokens': 802}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-78e4d2ee-d924-42bb-b834-9ebccd5495ae-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': ''}], usage_metadata={'input_tokens': 788, 'output_tokens': 14, 'total_tokens': 802})]}}
----
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', id='7f775265-fd10-45d1-b299-b5e95815e1c4', tool_call_id='')]}}
----
An error occurred: Error code: 400 - {'error': {'code': 'invalid_type', 'param': 'messages[0].content', 'message': "Invalid type for 'messages[0].content': expect

请注意，代理会执行多个查询，直到获得所需的信息：

- 列出可用表;
- 检索三个表的架构;
- 通过联接操作查询多个表。

然后，代理能够使用最终查询的结果来生成原始问题的答案。

代理可以类似地处理定性问题：

In [19]:
try:
    for s in agent_executor.stream(
        {"messages": [HumanMessage(content="Describe the playlisttrack table")]}
    ):
        print(s)
        print("----")
except Exception as e:
    print(f"An error occurred: {e}")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '', 'function': {'arguments': '{"table_names": "playlisttrack"}', 'name': 'sql_db_schema'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 786, 'total_tokens': 806}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f586f75e-0d41-4381-93e8-245544883823-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': ''}], usage_metadata={'input_tokens': 786, 'output_tokens': 20, 'total_tokens': 806})]}}
----
{'tools': {'messages': [ToolMessage(content="Error: table_names {'playlisttrack'} not found in database", name='sql_db_schema', id='41c5cca5-a325-442e-b3fc-aca67e6cb9dd', tool_call_id='')]}}
----
An error occurred: Error code: 400 - {'error': {'code': 'invalid_type', 'param': 'messages[0].content', 'message': "Invalid type for 'messages[0].content': expected

#### 处理高基数列

为了过滤包含专有名词（如地址、歌曲名称或艺术家）的列，我们首先需要仔细检查拼写，以便正确过滤数据。

我们可以通过创建一个包含数据库中存在的所有不同专有名词的向量存储来实现这一点。

然后，我们可以让代理在每次用户在他们的问题中包含专有名词时查询该向量存储，以找到该单词的正确拼写。

这样，代理可以确保在生成目标查询之前了解用户引用的实体。

首先，我们需要每个实体的唯一值，为此我们定义了一个函数，该函数将结果解析为元素列表：

In [21]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

['Heroes, Season',
 'Into The Light',
 'Machine Head',
 'Demorou...',
 'Album Of The Year']

使用此函数，我们可以创建一个检索器工具，代理可以自行决定执行。

In [23]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings

os.environ["DASHSCOPE_API_KEY"] = getpass.getpass() # 向量模型也是用阿里

vector_db = FAISS.from_texts(artists + albums, DashScopeEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """用于查找要筛选的值。input 是专有名词的近似拼写，输出是 
有效的专有名词。使用与搜索最相似的名词."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [24]:
print(retriever_tool.invoke("Alice Chains"))

Alice In Chains

Adrian Leaper & Doreen de Feis

Serie Sem Limite (Disc )

...And Justice For All

Hilary Hahn, Jeffrey Kahane, Los Angeles Chamber Orchestra & Margaret Batjer


这样，如果代理确定需要按照“Alice Chains”的思路编写基于艺术家的过滤器，则可以首先使用检索工具来观察列的相关值。

把这些放在一起：

In [25]:
system = """您是设计用于与 SQL 数据库交互的代理。
给定一个输入问题，创建一个语法正确的 SQLite 查询来运行，然后查看查询结果并返回答案。
除非用户指定了他们希望获取的特定数量的示例，否则请始终将查询限制为最多 5 个结果。
您可以按相关列对结果进行排序，以返回数据库中最有趣的示例。
永远不要查询特定表中的所有列，只要求给定问题的相关列。
您可以访问用于与数据库交互的工具。
仅使用给定的工具。仅使用工具返回的信息来构建您的最终答案。
在执行查询之前，必须仔细检查查询。如果在执行查询时遇到错误，请重写查询，然后重试。

不要对数据库进行任何 DML 语句（INSERT、UPDATE、DELETE、DROP 等）。

您可以访问以下表：{table_names}

如果你需要过滤一个专有名词，你必须总是首先使用“search_proper_nouns”工具查找过滤器值！
不要试图猜测正确的名称 - 使用此功能查找相似的名称。""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

agent = chat_agent_executor.create_tool_calling_executor(
    llm, tools, messages_modifier=system_message
)

In [26]:
try:
    for s in agent_executor.stream(
        {"messages": [HumanMessage(content="alis in chain有多少张专辑？")]}
    ):
        print(s)
        print("----")
except Exception as e:
    print(f"An error occurred: {e}")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 788, 'total_tokens': 802}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6e76275b-fdda-49a1-a720-c44998e17b9a-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': ''}], usage_metadata={'input_tokens': 788, 'output_tokens': 14, 'total_tokens': 802})]}}
----
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', id='639b3125-8f7a-41ea-bc28-49b8adffe1f0', tool_call_id='')]}}
----
An error occurred: Error code: 400 - {'error': {'code': 'invalid_type', 'param': 'messages[0].content', 'message': "Invalid type for 'messages[0].content': expect

正如我们所看到的，代理使用该工具来检查如何正确查询该特定艺术家的数据库。 `search_proper_nouns`